In [6]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

%matplotlib inline

Сгенерировать датасет при помощи sklearn.datasets.make_regression и обучить линейную модель при помощи градиентного спуска (GD) и (стохастического градиентного спуска (SGD) или mini-batch градиентного спуска). Построить графики среднеквадратичной ошибки от числа итераций для обоих методов на одном рисунке, сделать выводы о разнице скорости сходимости каждого из методов.

In [7]:
from sklearn import datasets

X, y, coef = datasets.make_regression(n_samples=100, 
                                      n_features=2, 
                                      n_informative=2, 
                                      n_targets=1,
                                      noise=5, 
                                      coef=True, 
                                      random_state=42)

In [10]:
n = X.shape[0]
alpha = 1e-3
w = np.array([1, 0.5])
grad = 1/n * 2 * np.dot(X.T, (np.dot(w, X.T) - y))
w - alpha * grad

array([1.1303913 , 0.64959218])